**N5GEH-Serv datamodeling workshop**
==========================================


A data model is an abstraction of a model that organizes elements of data and gives a standardization on properties and relationships. Furthermore, it provides a structured framework for understanding and organizing information within a database system, e.g. FIWARE stack. 

This example shows a possible workflow on how you can define or reuse use case specific data models and ensure FIWARE compatibility by merging these models with existing data model in FiLiP. The merged models can be used for interaction with FIWARE platform and in other information processing systems to establish interoperability.

# Use case

In the following example use case, we will capture data and information of multiple phasor measurement units (PMU)s in the FIWARE stack. Firstly, data models are created, then we will also show the process of exporting, sharing and reusing these data models. 

## Import dependencies

In [1]:
import os
os.chdir("/home/ubuntu/project/FiLiP")

from pydantic import ConfigDict, BaseModel
from pydantic.fields import Field, FieldInfo
from typing import Any, Optional
from pydantic_extra_types.coordinate import Coordinate
from datetime import datetime, date
import json 
from pprint import pprint

from filip.models import FiwareHeader
from filip.models.ngsi_v2.context import ContextEntityKeyValues
from filip.clients.ngsi_v2.cb import ContextBrokerClient



# Create data model

Good data modelling can harmonise representation formats and semantics for various applications. Therefore, we try to reuse existing data models. For our specific use case, we can also define our own data models.

## Common data model

Some data models are use case agnostic. For example, we reuse the data model of GeoCoordinates from [Schema.org](https://schema.org) to represent the location of PMUs.

In [2]:
class GeoCoordiantes(BaseModel):
    """
    https://schema.org/GeoCoordinates
    """
    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)  # Pydantic specific settings
    latitude: float = Field(
        alias="latitude",
        description="The latitude of a location. For example 37.42242 (WGS 84)."
    )
    longitude: float = Field(
        alias="longitude", 
        description="The longitude of a location. For example -122.08585 (WGS 84)."
    )
    address: Optional[str] = Field(
        alias="address",
        description="Physical address of the item.",
        default=None
    )
    addressCountry: Optional[str] = Field(
        alias="addressCountry",
        description="County code according to ISO 3166-1-alpha-2.",
        default=None
    )
    elevation: Optional[float] = Field(
        alias="elevation", 
        description="The elevation of a location (WGS 84). Values may be of the form 'NUMBER UNIT_OF_MEASUREMENT' (e.g., '1,000 m', '3,200 ft') while numbers alone should be assumed to be a value in meters.",
        default=None
    )
    postalCode: Optional[str] = Field(
        alias="postalCode", 
        description="The postal code. For example, 94043.",
        default=None
    )   

## Domain data models 

Some other data models already exist for specific domains. Sargon, for example, provides an open-source standardized semantic description for the electrical domain. 

In [3]:

class Angle(BaseModel):
    # model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/angle/
    """
    angle: float = Field(
        default=1.8,
        description="Quantity, measures angle in degree."
    ) 
class Frequency(BaseModel):
    model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/Frequency
    """
    frequency: float = Field(
        default=50.1,
        description="Quantity, measures frequency in Hz."
    )
class Magnitude(BaseModel):
    model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/Maqnitute
    """
    magnitude: float = Field(
        default=234329,
        description="Quantity, measures magnitude."
    )
class Rocof(BaseModel):
    model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/Rocf
    """
    rocof: float = Field(
        default=1,
        description="Quantity, measure rocof in Hz/s."
    )
class Timestamp(BaseModel):
    model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/Timestamp
    """
    timestamp: float = Field(
        default=8492394034,
        description="Quantity, time in s."
    )
     

In [4]:
class PMU(BaseModel):
    """
    https://sargon-n5geh.netlify.app/ontology/1.0/classes/PMU_
    """
    model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")  # Pydantic specific settings
    hasAngle: Any = Field(
        default=None,
        description="sargon:hasValue, link with a URI of Angle",
    )
    hasFrequency: Any = Field(
        default=None,
        description="sargon:hasValue, link with a URI of Frequency",
    ) 
    hasMagnitude: Any = Field(
        default=None,
        description="sargon:hasValue, link with a URI of Magnitude",
    )
    hasRocof: Any = Field(
        default=None,
        description="sargon:hasValue, link with a URI of Rocof",
    )
    hasTimestamp: Any = Field(
        default=None,
        description="sargon:hasValue, link with a URI of Timestamp",
    )
    location: GeoCoordiantes
 
          


## Compliance with FIWARE

After creating the data models, we also want to ensure the FIWARE compatibility. For this purpose, we can simply merge the data models of our use cases with the existing FIWARE models in FiLiP (>= 0.4.1).


In [5]:
class AngleFIWARE(Angle, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # Field info of the general FIWARE data model in FiLiP
    ContextEntityKeyValues.model_fields["type"],
    default="Angle"
)
class FrequencyFIWARE(Frequency, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # Field info of the general FIWARE data model in FiLiP
    ContextEntityKeyValues.model_fields["type"],
    default="Frequency"
)
class MagnitudeFIWARE(Magnitude, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # Field info of the general FIWARE data model in FiLiP
    ContextEntityKeyValues.model_fields["type"],
    default="Magnitude"
)
class RocofFIWARE(Rocof, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # Field info of the general FIWARE data model in FiLiP
    ContextEntityKeyValues.model_fields["type"],
    default="Rocof"
)
class TimestampFIWARE(Timestamp, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # Field info of the general FIWARE data model in FiLiP
    ContextEntityKeyValues.model_fields["type"],
    default="Timestamp"
)

     
class PMUFIWARE(PMU, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
    # First position is the field info of the parent class
    ContextEntityKeyValues.model_fields["type"],
    # set the default value
    default="CustomModels:PMU",
    # overwrite the title in the json-schema if necessary
    title="Type of the PMU",
    # overwrite the description
    description="https://sargon-n5geh.netlify.app/ontologies/Sargon.ttl",
    # validate the default value if necessary
    validate_default=True,
    # freeze the field if necessary
    frozen=True,
    # for more options see the pydantic documentation
    )

## Use the data model for your own application

Give the address of your context broker and change the environment variables SERVICE and SERVICE_PATH according to your settings in the entirety platform. 

In [6]:
CB_URL = "http://137.226.248.184:1026"
SERVICE = 'Test'
SERVICE_PATH = '/'
fiware_header = FiwareHeader(service=SERVICE,
                             service_path=SERVICE_PATH)
cb_client = ContextBrokerClient(url=CB_URL,
                                fiware_header=fiware_header)

Now we can export both the use case model and the FIWARE specific models to json-schema files and share it with other stakeholders that need the data.

In [7]:
use_case_model = PMUFIWARE.model_json_schema()
pprint(use_case_model)

{'$defs': {'GeoCoordiantes': {'description': 'https://schema.org/GeoCoordinates',
                              'properties': {'address': {'anyOf': [{'type': 'string'},
                                                                   {'type': 'null'}],
                                                         'default': None,
                                                         'description': 'Physical '
                                                                        'address '
                                                                        'of '
                                                                        'the '
                                                                        'item.',
                                                         'title': 'Address'},
                                             'addressCountry': {'anyOf': [{'type': 'string'},
                                                                          {'type': 'null'}],
 

Initialize entities.

In [8]:
entity_list = cb_client.get_entity_list()

pmu = PMUFIWARE(
    id="PMU:01",
    location=GeoCoordiantes(latitude=40.5, longitude=50.3)
)
angle = AngleFIWARE(
    id="Angle:01",
)
frequency = FrequencyFIWARE(
    id="Frequency:01",
)
magnitude = MagnitudeFIWARE(
    id="Magnitude:01",
)
rocof = RocofFIWARE(
    id="Rocof:01", 
)

timestamp = TimestampFIWARE(
    id="Timestamp:01",
)

Post entities, update the data and add the data to the PMU entity.

In [9]:
cb_client.post_entity(entity=pmu, key_values=True, update=True)
cb_client.post_entity(entity=angle, key_values=True, update=True)
cb_client.post_entity(entity=frequency, key_values=True, update=True)
cb_client.post_entity(entity=magnitude, key_values=True, update=True)



pprint(cb_client.get_entity_list())

pmu.hasAngle = angle.id
pmu.hasFrequency = frequency.id
pmu.hasMagnitude = magnitude.id
angle.angle = 25.3
pmu.location = GeoCoordiantes(longitude=30.4, latitude=50.4)
cb_client.update_entity_key_values(entity=angle)
cb_client.update_entity_key_values(entity=pmu)

pmu_data = cb_client.get_entity(entity_id="PMU:01", response_format="keyValues").model_dump()
print(pmu_data)
pmu_fiware = PMUFIWARE.model_validate(pmu_data)
# pmu = PMU.model_validate(pmu_data)


[ContextEntity(id='PMU:01', type='CustomModels:PMU', hasAngle=ContextAttribute(type='None', value=None, metadata={}), hasFrequency=ContextAttribute(type='None', value=None, metadata={}), hasMagnitude=ContextAttribute(type='None', value=None, metadata={}), hasRocof=ContextAttribute(type='None', value=None, metadata={}), hasTimestamp=ContextAttribute(type='None', value=None, metadata={}), location=ContextAttribute(type='StructuredValue', value={'latitude': 40.5, 'longitude': 50.3, 'address': None, 'addressCountry': None, 'elevation': None, 'postalCode': None}, metadata={})),
 ContextEntity(id='Angle:01', type='Angle', angle=ContextAttribute(type='Number', value=1.8, metadata={})),
 ContextEntity(id='Frequency:01', type='Frequency', frequency=ContextAttribute(type='Number', value=50.1, metadata={})),
 ContextEntity(id='Magnitude:01', type='Magnitude', magnitude=ContextAttribute(type='Number', value=234329.0, metadata={}))]
{'id': 'PMU:01', 'type': 'CustomModels:PMU', 'hasAngle': 'Angle:01


### Export serialized data models

In order to share data models, we usually need to export the Pydantic classes to serialized data format, i.e. JSON Schema in this example.


In [10]:
from pprint import pprint
import json
# Use case data model
pmu_schema = PMU.model_json_schema()
with open("./pmu_schema.json", "w") as f:
    json.dump(pmu_schema, f)
pprint(pmu_schema)

# FIWARE specific data models
pmu_fiware_schema = PMUFIWARE.model_json_schema()
with open("./pmu_fiware_schema.json", "w") as f:
    json.dump(pmu_fiware_schema, f)
pprint(pmu_fiware_schema)

{'$defs': {'GeoCoordiantes': {'description': 'https://schema.org/GeoCoordinates',
                              'properties': {'address': {'anyOf': [{'type': 'string'},
                                                                   {'type': 'null'}],
                                                         'default': None,
                                                         'description': 'Physical '
                                                                        'address '
                                                                        'of '
                                                                        'the '
                                                                        'item.',
                                                         'title': 'Address'},
                                             'addressCountry': {'anyOf': [{'type': 'string'},
                                                                          {'type': 'null'}],
 

In [11]:
angle_fiware_schema = AngleFIWARE.model_json_schema()
with open("./angle_fiware_schema.json", "w") as f:
    json.dump(angle_fiware_schema, f)
pprint(angle_fiware_schema)

{'additionalProperties': True,
 'properties': {'angle': {'default': 1.8,
                          'description': 'Quantity, measures angle in degree.',
                          'title': 'Angle',
                          'type': 'number'},
                'id': {'description': 'Id of an entity in an NGSI context '
                                      'broker. Allowed characters are the ones '
                                      'in the plain ASCII set, except the '
                                      'following ones: control characters, '
                                      'whitespace, &, ?, / and #.',
                       'example': 'Bcn-Welt',
                       'maxLength': 256,
                       'minLength': 1,
                       'title': 'Entity Id',
                       'type': 'string'},
                'type': {'default': 'Angle',
                         'description': 'Id of an entity in an NGSI context '
                                        'bro

## Use data models in other applications

As you can see above, the exported data model is based on normal JSON syntax, and is therefore both machine and human-readable. There are various libraries, that support parsing this data. For example, in Python jsonschema can be used to validate the data against the data model, and datamodel-code-generator can be used to generate python objects.


In [12]:
# Validation
from jsonschema import validate
pmu_data = cb_client.get_entity(entity_id="PMU:01", response_format="keyValues").model_dump()
validate(instance=pmu_data, schema=pmu_fiware_schema)
print("The data conforms to the data model.")

The data conforms to the data model.


In [13]:
# Parse serialized models
from pathlib import Path
from datamodel_code_generator import generate, DataModelType, InputFileType
output_path=Path("./pmu.py")
class_name="GeneratedPMU"
generate(Path("./pmu_fiware_schema.json"),
        class_name=class_name,
        output=output_path,
        output_model_type=DataModelType.PydanticV2BaseModel,
        use_subclass_enum=True,
        input_file_type=InputFileType.JsonSchema
        )
print(f"Python object '{class_name}' generated in '{output_path}'")

Python object 'GeneratedPMU' generated in 'pmu.py'


In [14]:
# Use generated python object
from pmu import GeneratedPMU
pmu_data = cb_client.get_entity(entity_id="PMU:01", response_format="keyValues").model_dump()
pmu_generated = GeneratedPMU.model_validate(pmu_data)
print(pmu_generated.model_dump_json(indent=2))
print(pmu_generated.location.model_dump_json(indent=2))

{
  "id": "PMU:01",
  "type": "CustomModels:PMU",
  "hasAngle": "Angle:01",
  "hasFrequency": "Frequency:01",
  "hasMagnitude": "Magnitude:01",
  "hasRocof": null,
  "hasTimestamp": null,
  "location": {
    "latitude": 50.4,
    "longitude": 30.4,
    "address": null,
    "addressCountry": null,
    "elevation": null,
    "postalCode": null
  }
}
{
  "latitude": 50.4,
  "longitude": 30.4,
  "address": null,
  "addressCountry": null,
  "elevation": null,
  "postalCode": null
}


#### Data model module in Entirety
The serialized data models can be used in `Entirety` as well. This section shows you, how `Entirety` can help you reuse data models and create data points in FIWARE stack. To reproduce this example, please make sure you use at least `Entirety` >= 1.2.0.

##### Load and save data model in Entirety
In `Entirety` open the data model module on the left side and open the window to create a new data model.
Enter on the top the name for your data model. Then copy the contents of a JSON Schema file that was created, e.g. the previous `angle_fiware_schema.json`, to the `Jsonschema` field as shown in the following figure. 

![Data model module](images/Datamodel_Entirety_1_1.png)

The text will be turned into the right format when you click on the button `Beautify Json` as can be seen in the next figure. 

![Data model module](images/Datamodel_Entirety_2_1.png)

Now the data model has just to be saved and is loaded in `Entirety`.

![Loaded data model](images/Datamodel_Entirety_3_1.png)

##### Create data points via data model
After loading data models in `Entirety`, entities can be created based on the data models. An example is showcased in the following figure.

![Entity creation via data model](images/Datamodel_Entirety_4_1.png)